In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Analysis and data wrangling for Ofcom broadband

In this analysis we are comparing different standard classifiers with the task of detecting whether a broadband connection belongs to an urban or a rural area. The models are trained and validated with the 2014 and 2015 datasets and they are tested in the 2016 dataset. 

The first step is to load both the 2014, 2015 and 2016 datasets into a proper format (DP). Notice that in some of the datasets we needed to discard some lines from the beginning of the files to locate the data

In [2]:
folder = '../../broadband_downloaded/'

data_2014_05 = pd.read_csv(folder + '2014_05/panellist_data.csv', encoding = "ISO-8859-1",header=3)
data_2014_11 = pd.read_csv(folder + '2014_11/panellist_data_november_2014.csv')
data_2015 = pd.read_csv(folder + '2015_11/panellist-data.csv')
data_2016 = pd.read_csv(folder + '2016_11/UK-home-broadband-performance,-November-2016-Panellist-data.csv',header=1)

At this stage, each table have a different format, with different features, same features with different names, etc. Since we want to concatenate the tables from 2014 and 2015, we need to agree on a common format for the integration (DI).

To do this, we only keep for each dataset the Id, Urban/rural, Download 24h, Upload 24h, Latency 24h and Web 24h features

In [3]:
#2014-05
features = ['ID', 'Urban/rural', 'Download speed (Mbit/s) 24 hrs', 'Upload speed (Mbit/s)24-hour',
       'Latency (ms)24-hour','Web page (ms)24-hour']
data_2014_05 = data_2014_05[features]

In [4]:
#2014-11
features = ['Id', 'Urban/rural', 'Download speed (Mbit/s) 24 hrs', 'Upload speed (Mbit/s)24-hour',
       'Latency (ms)24-hour','Web page (ms)24-hour']
data_2014_11 = data_2014_11[features]

In [5]:
#2015-11
features = ['unit_id', 'URBAN2', 'DL24hrmean', 'UL24hrmean', 
            'Latency24hr', 'Web24hr']
data_2015 = data_2015[features]

In [6]:
#2016-11
features = ['unit_id', 'Urban_Rural', 'trim24_mean_download', 'trim4_mean_upload',
            'latencyloss24','webget24']
data_2016 = data_2016[features]

In [7]:
#Rename headers of all tables
features = ['Id','Urban/Rural','Download speed','Upload speed','Latency','Web page']
data_2014_05.columns = features
data_2014_11.columns = features
data_2015.columns = features
data_2016.columns = features

In the 2015 dataset, they introduce the category 'semi-urban'. We have decided to keep it as is, disregarding that there might exist 'semi-urban' instances across 2014 or 2016 that were not encoded as such (NS).

We concatenate 2014_05, 2014_11 and 2015 into a single table, which is going to be the data used for training (DI)

In [8]:
X_train_df = pd.concat([data_2014_05,data_2014_11,data_2015],ignore_index=True)

Several Broadband devices were measured during several years, meaning that we have duplicates in our data that need to be removed according to the 'Id'. We will keep the newest recording in the training data (DI - Deduplication)

In [9]:
counts = X_train_df['Id'].value_counts()

In [10]:
X_train_unique_df = pd.DataFrame(columns=features)

for nn in range(X_train_df.shape[0]):
    if counts[X_train_df['Id'].iloc[nn]] > 1:
        counts[X_train_df['Id'].iloc[nn]] -= 1
    else:
        X_train_unique_df = X_train_unique_df.append(X_train_df.iloc[nn])

In [11]:
X_train_unique_df.shape

(3851, 6)

Remove possible duplicates from the test set (2016) of existing broadband devices in the training set (DI)

In [12]:
for elem in X_train_unique_df['Id']:
    data_2016.drop(data_2016[data_2016['Id'] == elem].index,inplace=True)

The training data has several missing entries in the urban/rural, latency and web page features, that correspond to less than 1% of the data. We decided to remove those entries from the analysis (MD)

In [13]:
index_nan = X_train_unique_df[X_train_unique_df['Urban/Rural'].isnull()].index
X_train_unique_df.drop(index_nan,inplace=True)

index_nan = X_train_unique_df[X_train_unique_df['Latency'].isnull()].index
X_train_unique_df.drop(index_nan,inplace=True)

index_nan = X_train_unique_df[X_train_unique_df['Web page'].isnull()].index
X_train_unique_df.drop(index_nan,inplace=True)

Transform the Urban/Rural feature into an ordinal variable (0: Rural, 1: Semi-urban, 2: Urban) (FE)

In [14]:
def urban_classes(x):
#     codes = dict({'Rural':0, 'Semi-urban':1, 'Urban':2})
    codes = dict({'Rural':0, 'Semi-urban':0, 'Urban':1})
    return codes[x]

X_train_unique_df['Urban/Rural'] = X_train_unique_df['Urban/Rural'].apply(urban_classes)
data_2016['Urban/Rural'] = data_2016['Urban/Rural'].apply(urban_classes)

Remove unnecessary features (Id), standardize the data and create X and y training and test for a multiclass logistic regression problem (Classify the broadband according to rural or urban based on the speeds of the devices) (FE)

In [15]:
# For classification problems
X_train_unique_df.drop(columns='Id',inplace=True)
data_2016.drop(columns='Id',inplace=True)

y_train = X_train_unique_df['Urban/Rural']
y_test = data_2016['Urban/Rural']

X_train_unique_df.drop(columns='Urban/Rural',inplace=True)
data_2016.drop(columns='Urban/Rural',inplace=True)

X_train_mean = X_train_unique_df.mean()
X_train_std = X_train_unique_df.std()
X_train = (X_train_unique_df - X_train_mean)/X_train_std
X_test = (data_2016 - X_train_mean)/X_train_std

# Classification comparison - with cross validation

In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import calibration_curve
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [18]:
#Setting parameters with grid search
lr = LogisticRegression(multi_class='auto')
gnb = GaussianNB()
svc = SVC()
rfc = RandomForestClassifier()

params_lr = {'penalty':['l2'], 'C':[0.1,1,10,100]}
parmas_gnb = {'priors':[None]}
params_svc = {'kernel':['linear','rbf'],'C':[0.1,1,10,100]}
params_rfc = {'max_depth':[None,2,3,4,10],'n_estimators':[10,100,200,500]}

classifiers = dict()
for clf, name, params in [(lr, 'Logistic', params_lr),
                  (gnb, 'Naive Bayes', parmas_gnb),
                  (svc, 'Support Vector Classification', params_svc),
                  (rfc, 'Random Forest', params_rfc)]:
    
    clf_grid = GridSearchCV(clf, params, cv=3)
    clf_grid.fit(X_train, y_train)
    
    #Select best model
    best_model = clf_grid.cv_results_['rank_test_score'].argmin()
    params_best_model = clf_grid.cv_results_['params'][best_model]
    
    #Test classifier
    clf.set_params(**params_best_model)
    
    classifiers[name] = clf
    
    print(name + ' parameters:')
    print(params_best_model)

Logistic parameters:
{'C': 0.1, 'penalty': 'l2'}
Naive Bayes parameters:
{'priors': None}
Support Vector Classification parameters:
{'C': 10, 'kernel': 'rbf'}
Random Forest parameters:
{'max_depth': 4, 'n_estimators': 100}


In [20]:
y_test_pos = np.argwhere(y_test.values==1)
y_test_neg = np.argwhere(y_test.values==0)

for name in classifiers:
    classifiers[name].fit(X_train, y_train)
    
    #Accuracy
    y_train_est = classifiers[name].predict(X_train)
    y_test_est = classifiers[name].predict(X_test)
    
    #Precision - recall - f1 score
    TP = sum((y_test.values==y_test_est)[y_test_pos])[0]
    TN = sum((y_test.values==y_test_est)[y_test_neg])[0]
    FP = sum((y_test.values!=y_test_est)[y_test_neg])[0]
    FN = sum((y_test.values!=y_test_est)[y_test_pos])[0]
    
    prec = TP/(TP+FP)
    rec = TP/(TP+FN)
#     f1 = 2/(1/prec + 1/rec)
    
    print(name + ' train accuracy: ' + str((y_train_est == y_train).mean().round(4)))
    print(name + ' train F1: ' + str(f1_score(y_train,y_train_est).round(4)))
    print(name + ' test accuracy: ' + str((y_test_est == y_test).mean().round(4)))
    print(name + ' test F1: ' + str(f1_score(y_test,y_test_est).round(4)))
    print(name + ' test precision: ' + str(prec.round(4)))
    print(name + ' test recall: ' + str(rec.round(4)))
    
    print('Confusion matrix train: ')
    print(confusion_matrix(y_train,y_train_est))
    
    print('Confusion matrix test: ')
    print(confusion_matrix(y_test,y_test_est))

Logistic train accuracy: 0.6975
Logistic train F1: 0.8039
Logistic test accuracy: 0.7764
Logistic test F1: 0.8582
Logistic test precision: 0.7933
Logistic test recall: 0.9347
Confusion matrix train: 
[[ 291  965]
 [ 168 2322]]
Confusion matrix test: 
[[ 275  485]
 [ 130 1861]]
Naive Bayes train accuracy: 0.6925
Naive Bayes train F1: 0.7945
Naive Bayes test accuracy: 0.7666
Naive Bayes test F1: 0.8477
Naive Bayes test precision: 0.8031
Naive Bayes test recall: 0.8975
Confusion matrix train: 
[[ 367  889]
 [ 263 2227]]
Confusion matrix test: 
[[ 322  438]
 [ 204 1787]]
Support Vector Classification train accuracy: 0.705
Support Vector Classification train F1: 0.8134
Support Vector Classification test accuracy: 0.7644
Support Vector Classification test F1: 0.8548
Support Vector Classification test precision: 0.7718
Support Vector Classification test recall: 0.9578
Confusion matrix train: 
[[ 232 1024]
 [  81 2409]]
Confusion matrix test: 
[[ 196  564]
 [  84 1907]]
Random Forest train acc